# PREVIOUS

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()



# from huggingface_hub import HfApi

# user_id = HfApi().whoami()["name"]

# print(f"user id '{user_id}' will be used during the example")



# from datasets import concatenate_datasets, load_dataset

# bookcorpus = load_dataset("bookcorpus", split="train")
# wiki = load_dataset("wikipedia", "20220301.en", split="train")
# wiki = wiki.remove_columns([col for col in wiki.column_names if col != "text"])  # only keep the 'text' column

# assert bookcorpus.features.type == wiki.features.type
# raw_datasets = concatenate_datasets([bookcorpus, wiki])

In [1]:
!pip install transformers datasets
!sudo apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
from datasets import Dataset
import json
from tqdm import tqdm
from transformers import BertTokenizerFast, AutoTokenizer, AutoModelForMaskedLM
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [1]:
# load file
with open('/content/IndiFinBERT_TextMapping.json', 'r') as f:
  data = json.load(f)

# create a dataset with each row containing 500 characters
texts = []
ori_text = data['76950']
for i in range(len(ori_text)):
  if i % 500 == 0:
    texts.append({'text': ori_text[i: i+500]})
raw_dataset = Dataset.from_list(texts)


# repositor id for saving the tokenizer
tokenizer_id="bert-base-uncased-trial"

# create a python generator to dynamically load the data
def batch_iterator(batch_size=10):
    for i in tqdm(range(0, len(raw_dataset), batch_size)):
        yield raw_dataset[i : i + batch_size]["text"]

# create a tokenizer from existing one to re-use special tokens
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# train new tokenizer
new_tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32000)
# new_tokenizer.save_pretrained("tokenizer")

# Get the vocabulary as a dictionary
vocab_dict = tokenizer.vocab
# Print all tokens in the vocabulary
for token, token_id in vocab_dict.items():
    print(f"Token: {token}, ID: {token_id}")


# import model
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")
# check number of parameters of model
num_params = sum(p.numel() for p in model.parameters())
num_params

# get a list of embeddings from the model
def get_embeddings(tokenizer, model):
    embeddings = []
    print(f"Vocab size: {tokenizer.vocab_size}")
    i = 0
    for token in tokenizer.vocab:
        i += 1
        if i % 100 == 0:
            print(f"Processed {i} out of {tokenizer.vocab_size} tokens")
        inputs = tokenizer(token, return_tensors='pt')
        outputs = model(**inputs, output_hidden_states=True)

        # Access the last hidden state from the tuple of hidden states
        last_hidden_state = outputs.hidden_states[-1]

        # Typically, you would use the first token's embedding (e.g., [CLS] token)
        # But since you're processing single tokens, you might want to use the only token available
        embedding = last_hidden_state[:, 0, :].detach().numpy()[0]

        embeddings.append(embedding)
    return embeddings

# embed the pre-trained vocabulary
print("starting first embedding")
embeddings = get_embeddings(tokenizer, model)
# embed the new vocabulary
print("starting second embedding")
new_embeddings = get_embeddings(new_tokenizer, model)


# Calculate cosine similarity
cosine_similarities = []
for new_embedding, embedding in zip(new_embeddings, embeddings):
    similarity = cosine_similarity([new_embedding], [embedding])[0][0]
    cosine_similarities.append(similarity)

# Print average cosine similarity
average_similarity = np.mean(cosine_similarities)
print(f"Average Cosine Similarity: {average_similarity}")


# save them in a json as a list of lists
array_lists = [array.tolist() for array in new_embeddings]
with open('new_arrays.json', 'w') as f:
    json.dump(array_lists, f, indent=4)

hello


# OTHER SHIT

In [15]:
def get_embeddings(tokenizer, model):
    embeddings = []
    for token in tokenizer.vocab:
        inputs = tokenizer(token, return_tensors='pt')
        outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state[:, 0, :].detach().numpy()[0])
    return embeddings

embeddings = get_embeddings(tokenizer, model)
# new_embeddings = get_embeddings(new_tokenizer, model)

# Calculate cosine similarity
cosine_similarities = []
for new_embedding, embedding in zip(new_embeddings, embeddings):
    similarity = cosine_similarity([new_embedding], [embedding])[0][0]
    cosine_similarities.append(similarity)

# Print average cosine similarity
average_similarity = np.mean(cosine_similarities)
print(f"Average Cosine Similarity: {average_similarity}")

AttributeError: 'MaskedLMOutput' object has no attribute 'last_hidden_state'

# TESTING

In [28]:
from transformers import BertTokenizerFast

In [29]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [33]:
# prompt: find vocab length of tokenizer

len(tokenizer.vocab)

30522

In [ ]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

In [ ]:
model.config

BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.48.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
# prompt: find parameter size of model

import torch

def get_parameter_sizes(model):
  """
  Prints the sizes of all parameters in a PyTorch model.
  """
  total_params = 0
  for name, param in model.named_parameters():
    param_size = param.nelement() * param.element_size() / 1024**2 # in MB
    print(f"{name}: {param.size()}, size: {param_size:.3f} MB")
    total_params += param.nelement()
  print(f"\nTotal number of parameters: {total_params}")
  print(f"Total size of parameters: {total_params * 4 / 1024**2:.3f} MB") # Assuming float32 (4 bytes)

get_parameter_sizes(model)


embeddings.word_embeddings.weight: torch.Size([30522, 128]), size: 14.903 MB
embeddings.position_embeddings.weight: torch.Size([512, 128]), size: 0.250 MB
embeddings.token_type_embeddings.weight: torch.Size([2, 128]), size: 0.001 MB
embeddings.LayerNorm.weight: torch.Size([128]), size: 0.000 MB
embeddings.LayerNorm.bias: torch.Size([128]), size: 0.000 MB
encoder.layer.0.attention.self.query.weight: torch.Size([128, 128]), size: 0.062 MB
encoder.layer.0.attention.self.query.bias: torch.Size([128]), size: 0.000 MB
encoder.layer.0.attention.self.key.weight: torch.Size([128, 128]), size: 0.062 MB
encoder.layer.0.attention.self.key.bias: torch.Size([128]), size: 0.000 MB
encoder.layer.0.attention.self.value.weight: torch.Size([128, 128]), size: 0.062 MB
encoder.layer.0.attention.self.value.bias: torch.Size([128]), size: 0.000 MB
encoder.layer.0.attention.output.dense.weight: torch.Size([128, 128]), size: 0.062 MB
encoder.layer.0.attention.output.dense.bias: torch.Size([128]), size: 0.000 MB